In [1]:
import astropy.units as u
import astropy.constants as const
import numpy as np

# Use to obtain data
import pandas as pd
import urllib
# the service URL
livechart = "https://nds.iaea.org/relnsd/v1/data?"

# There have been cases in which the service returns an HTTP Error 403: Forbidden
# use this workaround
import urllib.request
def lc_pd_dataframe(url):
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req))

In [2]:
# Example
df=lc_pd_dataframe(livechart+"fields=ground_states&nuclides=235U")
df

,z,n,symbol,radius,unc_r,abundance,unc_a,energy_shift,energy,unc_e,...,unc_ba,atomic_mass,unc_am,massexcess,unc_me,me_systematics,discovery,ENSDFpublicationcut-off,ENSDFauthors,Extraction_date
0,92,143,U,5.8337,0.0041,0.7204,0.0006,NaN,0,NaN,...,0.0048,2.350439e+08,1.198,40918.782,1.116,N,1935,1-Feb-2014,E. BROWNE and J. K. TULI,2024-05-29


## Question 1

Assume ${\rm ^{235}U}$ splits into two fragments with masses $100$ and $132$ and three neutrons. Find the masses of stable nuclides with these masses. What is the fission energy released by the fission of $1.0\,{\rm kg}$ of ${\rm ^{235}U}$?

### Answer

Through data from [Live Chart of Nuclides](https://www-nds.iaea.org/relnsd/vcharthtml/VChartHTML.html), we can see that The stable nuclide with mass $100$ is ${\rm ^{100}_{ ~~44} Ru}$ and the stable nucilde with mass $132$ is ${\rm ^{132}_{ ~~54} Xe}$.

So the fission process is

$$
^{235}_{~~92}\!{\rm U} \to ^{100}_{ ~~44}\!{\rm Ru} + ^{132}_{ ~~54}\!{\rm Xe} + 3n +6e
$$

The energy of the fission process is

$$
\Delta E = m_{\rm ^{235}U} - m_{\rm ^{100}Ru} - m_{\rm ^{132}Xe} - 3m_n - 6m_e
$$

In [17]:
m_235U = lc_pd_dataframe(livechart+"fields=ground_states&nuclides=235U")['atomic_mass'][0]*1e-6*u.u
m_100Ru = lc_pd_dataframe(livechart+"fields=ground_states&nuclides=100Ru")['atomic_mass'][0]*1e-6*u.u
m_132Xe = lc_pd_dataframe(livechart+"fields=ground_states&nuclides=132Xe")['atomic_mass'][0]*1e-6*u.u
m_n = lc_pd_dataframe(livechart+"fields=ground_states&nuclides=1n")['atomic_mass'][0]*1e-6*u.u
m_e = (const.m_e).to('u')

In [28]:
Delta_m = m_235U - (6*m_e + 3*m_n + m_100Ru + m_132Xe)
Delta_E = (Delta_m*const.c**2).to("MeV")
Delta_E

<Quantity 192.14520013 MeV>

Thus the energy released per reaction is $\Delta E = 192\,{\rm MeV}$. By the fission of $1.0\,{\rm kg}$ of ${\rm ^{235}\!U}$, the number of fission reaction is $N$ where

$$
N = \frac{ \rm 1.0\,kg}{m_{\rm ^{235}\!U}}
$$

In [33]:
N = (1*u.kg/m_235U).to(' ') # Dimensionless
N

<Quantity 2.56213416e+24>

Thus $N = 2.56\times 10^{24}$, the energy released is $E = N\cdot \Delta E$

In [38]:
E = (N*Delta_E).to('J')
E

<Quantity 7.88754409e+13 J>

Thus the energy released by $1.0 \,{\rm kg}$ of ${\rm ^{235}U}$ through this reaction is $7.89\times 10^{13}\,{\rm J}$.

## Question 2

If a natural uranium, thermal fission reactor is operating at a thermal power output level of $\rm 2\,GW$. Calculate the total rate of consumption of $\rm ^{235}U$(in $\rm kg\cdot yr^{-1}$). Take the energy release per fission to be $E = {\rm 200\,MeV}$.

### Answer

The power of the reactor is $W = 2\,\rm{GW}$ and the energy realsed per fission is $E = 200\,{\rm MeV}$. Thus there are $n$ times of fissions per second.

$$
n = \frac{W}{E}
$$

In [10]:
W = 2*u.GW
E = 200*u.MeV
n = (W/E).si
n

<Quantity 6.24150907e+19 1 / s>

Thus $n = 6.241\times 10^{19} \,{\rm s^{-1}}$.

So the consumption rate of the number of $\rm ^{235}U$ is $n$. The comsumption rate of the mass of $\rm ^{238}U$ is $n\cdot m_{\rm ^{235}U}$ where $m_{\rm ^{235}U}$ is the atomic mass of $\rm ^{235}U$.

In [15]:
R = n*(235*u.u)
R.to('kg/yr')

<Quantity 768.61796858 kg / yr>

So the consumption rate is
$$
R = 768\,{\rm kg/yr}
$$

## Question 3

What is the meaning of delayed neutrons? What is the importance of delayed neutrons in a nuclear reactor?

### Answer

Some neutrons produced in fission are delayed by some time as they are produced in subsequent decays of the initial fission fragments. These are called delayed neutrons. 

This situation is crucial to mechanical control of the reactor. If all the fission neutrons were produced instantly in fission, there would be no time for the minute adjustment required in a reactor to keep it critical.

## Question 4

Describe some design features of a pressurized water reactor.

### Answer (Accroding to [https://www.mhi.com/products/energy/light_water_reactor_plant_features.html](https://www.mhi.com/products/energy/light_water_reactor_plant_features.html))

The basic design concept of the PWR nuclear power plant is simple: pressurized water is used as the reactor coolant and the reactor and turbine systems are separated by steam generators to constitute an indirect cycle system. The outstanding features of this system are shown in the following:

- The higher moderator density makes it possible to minimize the fuel rod pitch and enable a compact reactor core design.
- The absence of bulk boiling in the core limits cladding corrosion and thereby increases the fuel reliability.
The open-lattice fuel assembly design contributes to the economical use of neutrons and results in excellent fuel economy.
- The self-control mechanisms are inherent, and a safe shutdown capability is ensured by gravity insertion of control rods in an emergency.
- The plant generates an extremely low quantity of radioactive substance.
- The reactor and turbine systems are separated by steam generators; therefore, the turbine system design of the combustion power plant grade is feasible and thereby it can be easily operated and maintained.
- The core cooling with heat-removal capability of the secondary system ensures inherent reactor safety in an emergency.
- The lower center-of-gravity plant design favors seismic resistance.

As shown above, since the PWR nuclear power plant enables to design its facilities compactly as a whole plant and provides superior fuel economy, thus enables to sufficiently respond to any needs of nuclear power plants such as inexpensive generation root and excellent operability.